In [1]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('id3.csv')
df

,day,outlook,temperature,humidity,wind,playtennis
0,1,sunny,hot,high,weak,no
1,2,sunny,hot,high,strong,no
2,3,overcast,hot,high,weak,yes
3,4,rain,mild,high,weak,yes
4,5,rain,cool,normal,weak,yes
5,6,rain,cool,normal,strong,no
6,7,overcast,cool,normal,strong,yes
7,8,sunny,mild,high,weak,no
8,9,sunny,cool,normal,weak,yes
9,10,rain,mild,normal,weak,yes


In [4]:
import csv
f = open('id3.csv','r')
data = list(csv.reader(f))
for i in data:
    print(i)

['day', 'outlook', 'temperature', 'humidity', 'wind', 'playtennis']
['1', 'sunny', 'hot', 'high', 'weak', 'no']
['2', 'sunny', 'hot', 'high', 'strong', 'no']
['3', 'overcast', 'hot', 'high', 'weak', 'yes']
['4', 'rain', 'mild', 'high', 'weak', 'yes']
['5', 'rain', 'cool', 'normal', 'weak', 'yes']
['6', 'rain', 'cool', 'normal', 'strong', 'no']
['7', 'overcast', 'cool', 'normal', 'strong', 'yes']
['8', 'sunny', 'mild', 'high', 'weak', 'no']
['9', 'sunny', 'cool', 'normal', 'weak', 'yes']
['10', 'rain', 'mild', 'normal', 'weak', 'yes']
['11', 'sunny', 'mild', 'normal', 'strong', 'yes']
['12', 'overcast', 'mild', 'high', 'strong', 'yes']
['13', 'overcast', 'hot', 'normal', 'weak', 'yes']
['14', 'rain', 'mild', 'high', 'strong', 'no']


In [11]:
df.drop('day',axis=1,inplace=True)


KeyError: "['day'] not found in axis"

In [12]:
df

,outlook,temperature,humidity,wind,playtennis
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cool,normal,weak,yes
5,rain,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes
9,rain,mild,normal,weak,yes


In [13]:
attributeNames = list(df.columns)

In [14]:
attributeNames

['outlook', 'temperature', 'humidity', 'wind', 'playtennis']

In [17]:
attributeNames.remove('playtennis')
attributeNames

['outlook', 'temperature', 'humidity', 'wind']

In [18]:
label = 'playtennis'

In [32]:
def cal_total_entropy(train_data,label,classlist):
    total_rows = train_data.shape[0]
    totalentropy = 0
    for c in classlist:
        totalclasscount = train_data[train_data[label] == c].shape[0]
        
        totalclassent = -(totalclasscount/total_rows) * np.log2(totalclasscount/total_rows)
        totalentropy += totalclassent
    
    return totalentropy
    

In [33]:
def calc_entropy(feature_value_data,label,classlist):
    classcount = feature_value_data.shape[0]
    entropy = 0
    for c in classlist:
        
        labelclasscount = feature_value_data[feature_value_data[label] == c].shape[0]
        
#         totalclassent = -(totalclasscount/total_rows) * np.log2(totalclasscount/total_rows)
#         totalentropy += totalclassent
        class_entropy = 0
        if labelclasscount != 0:
            class_entropy = -(labelclasscount/classcount) * np.log2(labelclasscount/classcount)
            entropy += class_entropy
        
    return entropy
        
    

In [34]:
def clac_info_gain(train_data,feature_name,label,classlist):
    feature_value_list = train_data[feature_name].unique()
    total_rows = train_data.shape[0]
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value]
        feature_value_data_count = feature_value_data.shape[0]
        
        feature_value_ent = calc_entropy(feature_value_data,label,classlist)
        
        feature_value_prob = feature_value_data_count/total_rows
        feature_info += feature_value_prob * feature_value_ent
        
    return cal_total_entropy(train_data,label,classlist) - feature_info

In [35]:

def id3(df, target_attribute_name, attribute_names, default_class = None):
    from collections import Counter
    count = Counter(x for x in df[target_attribute_name])
    if len(count)==1:
        return next(iter(count))
    elif df.empty or (not attribute_names):
        return default_class
    else:
        default_class = max(count.keys())
        class_list = df[label].unique()
        gain = [
            clac_info_gain(df, attr, target_attribute_name,class_list) for attr in attribute_names
        ]
        print()
        index_of_max = gain.index(max(gain))
        best_attr = attribute_names[index_of_max]
        
        tree = {best_attr:{}}
        
        remaining_attribute_names = [ i for i in attribute_names if i!= best_attr ]
        
        for attr_val, data_subset in df.groupby(best_attr):
                subtree = id3(data_subset, target_attribute_name, remaining_attribute_names, default_class)
                tree[best_attr][attr_val] = subtree
        
        return tree

tree = id3(df,'playtennis',attributeNames)
print("\n\nThe Resultant Decision Tree is:\n")
print(tree)






The Resultant Decision Tree is:

{'outlook': {'overcast': 'yes', 'rain': {'wind': {'strong': 'no', 'weak': 'yes'}}, 'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}
